In [2]:
import spacy
import numpy as np
from annoy import AnnoyIndex

In [3]:
nlp = spacy.load('en')

In [4]:
# from https://github.com/aparrish/plot-to-poem/blob/master/plot-to-poem.ipynb
def meanvector(text):
    s = nlp(text)
    vecs = [word.vector for word in s \
            if word.pos_ in ('NOUN', 'VERB', 'ADJ', 'ADV', 'PROPN', 'ADP') \
            and np.any(word.vector)] # skip all-zero vectors
    if len(vecs) == 0:
        raise IndexError
    else:
        return np.array(vecs).mean(axis=0)
    
meanvector("this is a blah blah what do you want").shape

(384,)

In [5]:
t = AnnoyIndex(384, metric='angular')

In [6]:
i = 0
file = open("dinner_talks_sample", "r")
lines = file.readlines()
inserted_lines = list()
for line in lines:
    line = line.rstrip()    
    try:        
        mean_vector = meanvector(line)        
        t.add_item(i, mean_vector)
        inserted_lines.append(line)
        i += 1
    except IndexError:
        print("Index error!")
        continue        

In [7]:
t.build(100)

True

In [8]:
print("{} items in vector space for {} lines".format(t.get_n_items(), len(inserted_lines)))
assert(t.get_n_items() == len(lines))

50 items in vector space for 50 lines


In [18]:
nearest = t.get_nns_by_vector(meanvector("You're talking to me?"), n=5, include_distances=True)
print(nearest)

([48, 10, 28, 24, 34], [0.0, 0.832426905632019, 0.8468745350837708, 0.9303187131881714, 0.9437580704689026])
